In [1]:
import azure.functions as func
import requests
from datetime import datetime, timedelta
import pytz

import pandas as pd
import json
import logging
import os
import traceback
import time

In [2]:
from log import setup_logger, thread_local
from database import MySql, AzureStorage

'2024-06-06-eap'

In [72]:
data_type = 'eap'
logger = setup_logger(data_type)
logger.info('===== Chat log data summary start!')
as_instance_log = AzureStorage(container_name='logs', storage_type='datast')

In [ ]:
corp_cd_list = ['LGEDG']
start_creation_date_utc = '20240430'

try :

    start_utc_now = datetime.now(pytz.utc)
    input_timestamp_utc = start_utc_now.strftime("%Y-%m-%dT%H:%M:%S")

    logger.info(f"input : {input_timestamp_utc}")
    logger.info(f"start : {start_creation_date_utc}")
    #logger.info(f"end : {end_creation_date_utc}")
    
    # url = 'https://sqslec61id.execute-api.ap-northeast-2.amazonaws.com/postDailyContact'
    url = 'https://4rz1xg9u5b.execute-api.ap-northeast-2.amazonaws.com/postDailyContactProd'

    for corp_cd in corp_cd_list:

        parameters = (input_timestamp_utc, corp_cd, start_creation_date_utc, end_creation_date_utc)

        logger.info(f"############## CORP_CD : {corp_cd}")
        logger.info(f"############## TIMESTAMP : {input_timestamp_utc}")
        logger.info(f"############## START : {start_creation_date_utc}")
        logger.info(f"############## END : {end_creation_date_utc}")

        query = """
                select D.EAP_CORP_CD 												AS corporation_code
                    , D.CREATION_DATE 												AS creation_date
                    , 'C05' 														AS contact_channel_code
                    , 'Chatbot'														AS contact_channel_name
                    , 'Chatbot'														AS system_name
                    , 'Chatbot'														AS input_user_id
                    , %s					                                        AS input_timestamp
                    -- , CAST(D.TOTAL_COUNT AS DOUBLE) 								AS total_count -- 주석 컬럼
                    -- , CAST(D.TOTAL_EXCLUDE_SESSION AS DOUBLE) 						AS total_exclude_session -- 주석 컬럼
                    -- , CAST(D.LIVECHAT_SESSION AS DOUBLE) 							AS livechat_session -- 주석 컬럼
                    , CAST(D.TOTAL_EXCLUDE_SESSION - D.LIVECHAT_SESSION AS DOUBLE) 	AS inbound_count
                    -- , NULL					                						AS outbound_count
                    , CAST(D.TOTAL_EXCLUDE_SESSION - D.LIVECHAT_SESSION AS DOUBLE) 	AS answered_count
                    -- , NULL								                			AS sum_answer_speed
                    , CAST(D.SUM_TALK_TIME AS DOUBLE) 								AS sum_talk_time
                    -- , CAST(D.SUM_TALK_TIME / D.TOTAL_COUNT AS DOUBLE) 				AS avg_talk_time -- 주석 컬럼
                    -- , NULL									                		AS sum_call_work
                    -- , NULL									                		AS sum_hold_time
                    , CAST(D.SURVEY_SOLVED_SESSION AS DOUBLE) 						AS sum_q1_answer
                    , CAST(D.SURVEY_SESSION AS DOUBLE) 								AS sum_q1_response
                    -- , CAST(D.SURVEY_SOLVED_SESSION / D.SURVEY_SESSION AS DOUBLE) 	AS counseling_complet_rate -- 주석 컬럼
                    , CAST(D.SUM_CHAT_RATING AS DOUBLE) 							AS sum_q2_answer
                    , CAST(D.SURVEY_SESSION2 AS DOUBLE) 							AS sum_q2_response
                    -- , CAST(D.SUM_CHAT_RATING / D.SURVEY_SESSION2 AS DOUBLE) 		AS satisfaction_score -- 주석 컬럼
                    -- , NULL              											AS sum_waiting_time
                    -- , NULL			                								AS repeat_call_count
                    -- , NULL							                				AS last_agent_conn_count
                    -- , NULL					                						AS Ivr_call_count
                    -- , NULL								                			AS answered_20s_count
                    -- , NULL						                					AS auto_unmanned_ivr_count
                    -- , NULL								                			AS agent_login_count
                    -- , NULL							                				AS agent_inbound_count
                    -- , NULL								                			AS agent_outbound_count
                    -- , NULL								                			AS agent_inbound_outbound_count
                    -- , NULL								                			AS agent_inbound_lt_5call_count
                from (
                    select C.EAP_CORP_CD
                        , C.CREATION_DATE
                        , COUNT(C.CHAT_ID) AS TOTAL_COUNT
                        , SUM(CASE WHEN C.TYPE_SEESION_YN = 'Y' THEN 1 ELSE 0 END) AS TOTAL_EXCLUDE_SESSION
                        , SUM(CASE WHEN C.TYPE_LIVECHAT_YN = 'Y' THEN 1 ELSE 0 END) AS LIVECHAT_SESSION
                        , SUM(C.CHAT_TIME) AS SUM_TALK_TIME
                        , SUM(CASE WHEN C.TYPE_SURVEY_YN = 'Y' THEN 1 ELSE 0 END) AS SURVEY_SESSION
                        , SUM(CASE WHEN C.TYPE_SOLVED_YN = 'Y' THEN 1 ELSE 0 END) AS SURVEY_SOLVED_SESSION
                        , SUM(C.CHAT_RATING) AS SUM_CHAT_RATING
                        , SUM(CASE WHEN C.CHAT_RATING > 0 THEN 1 ELSE 0 END) AS SURVEY_SESSION2
                    from (
                        select A.*, B.ATTRIBUTE1 as EAP_CORP_CD, DATE_FORMAT(A.CHAT_DT, '%Y%m%d') as CREATION_DATE
                        from tb_chat_session_info A
                        left join tb_code_mst B
                        on 1=1
                            and B.GROUP_CD = 'B00008'
                            and A.COUNTRY_CD = B.CODE_CD
                    ) C
                    group by C.EAP_CORP_CD, C.CREATION_DATE
                    having 1=1
                    -- and C.EAP_CORP_CD IS NOT NULL -- 전체 조회 조건
                    and C.EAP_CORP_CD =  %s -- 조회조건
                    and C.CREATION_DATE >= %s
                ) D
                order by 1, 2;
                """
        
        sql_instajce = MySql()
        df = sql_instajce.get_table(query=query, parameters=parameters)
        df = pd.DataFrame(df)
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

        logger.info(f"############## LEN : {len(df)}")
        
        for index, item in df.iterrows():

            data = json.loads(item.to_json())
            body = {"auth_code": "S10",
                "auth_key": "5770abd8c81fc290a60bd95313b4a258ad9714eb4090fd7d3da079aeca64f264b2164ca28856d82c3da2023b0e50b50b88515fa6eb9e25fa866c7425bbf480ad",
                "data": data
                }
            
            logger.info(json.dumps(body, indent=4))
            
            headers = {
            'Content-Type': 'application/json',
            }

            response = requests.post(url, json=body, headers=headers)

            # 응답 확인
            if response.status_code == 200:
                logger.info(f'Success : {response.json()}')
            else:
                logger.error(f'Failed : {response.status_code}, {response.text}')

except Exception as e :
    error = traceback.format_exc()
    logger.error(error)

filter_logs_data_type = filter_logs_by_data_type(logs=thread_local.log_list, data_type=data_type)
logs_str = "\n".join(filter_logs_data_type)

formatted_date = datetime.now().strftime('%Y-%m-%d')
log_file_path = f"eapapi/{formatted_date}/eap.log"
as_instance_log.upload_log(logs_str, file_path=log_file_path)

In [94]:
sql_instajce = MySql()
corp_mst = sql_instajce.get_table(query = """
                                SELECT * FROM TB_CODE_MST
                                WHERE GROUP_CD = 'B00008' AND USE_YN = 'Y';
                                """)
corp_mst = pd.DataFrame(corp_mst)
corp_mst = corp_mst.applymap(lambda x: x.strip() if isinstance(x, str) else x)
corp_cd_list = corp_mst['ATTRIBUTE1'].unique().tolist()

start_utc_now = datetime.now(pytz.utc)
input_timestamp_utc = start_utc_now.strftime("%Y-%m-%dT%H:%M:%S")

start_creation_date_utc = (start_utc_now-timedelta(days=1)).strftime("%Y%m%d")
end_creation_date_utc = start_utc_now.strftime("%Y%m%d")

logger.info(f"input : {input_timestamp_utc}")
logger.info(f"start : {start_creation_date_utc}")
logger.info(f"end : {end_creation_date_utc}")

# url = 'https://sqslec61id.execute-api.ap-northeast-2.amazonaws.com/postDailyContact'
# url = 'https://4rz1xg9u5b.execute-api.ap-northeast-2.amazonaws.com/postDailyContactProd'

In [95]:
corp_cd_list

['LGEAG', 'LGEDG', 'LGESW', 'LGEFS', 'LGEIS', 'LGEUK']

In [92]:
open_info = {'LGEDG' : '20240430',
             'LGEES' : '20240605',
             'LGEFS' : '20240524',
             'LGEIS' : '20240430',
             'LGESW' : '20240531',
             'LGEUK' : '20240529',
             'LGEAG' : '20240430'}

In [93]:
start_utc_now = datetime.now(pytz.utc)
input_timestamp_utc = start_utc_now.strftime("%Y-%m-%dT%H:%M:%S")

# url = 'https://sqslec61id.execute-api.ap-northeast-2.amazonaws.com/postDailyContact'
url = 'https://4rz1xg9u5b.execute-api.ap-northeast-2.amazonaws.com/postDailyContactProd'

for corp_cd in corp_cd_list :
    start_creation_date_utc = open_info[corp_cd]
    end_creation_date_utc = '20240604'

    parameters = (input_timestamp_utc, corp_cd, start_creation_date_utc, end_creation_date_utc)

    print(f"############## CORP_CD : {corp_cd}")
    print(f"############## TIMESTAMP : {input_timestamp_utc}")
    print(f"############## START : {start_creation_date_utc}")
    print(f"############## END : {end_creation_date_utc}")

    query = """
            select D.EAP_CORP_CD 												AS corporation_code
                , D.CREATION_DATE 												AS creation_date
                , 'C05' 														AS contact_channel_code
                , 'Chatbot'														AS contact_channel_name
                , 'Chatbot'														AS system_name
                , 'Chatbot'														AS input_user_id
                , %s					                                        AS input_timestamp
                -- , CAST(D.TOTAL_COUNT AS DOUBLE) 								AS total_count -- 주석 컬럼
                -- , CAST(D.TOTAL_EXCLUDE_SESSION AS DOUBLE) 						AS total_exclude_session -- 주석 컬럼
                -- , CAST(D.LIVECHAT_SESSION AS DOUBLE) 							AS livechat_session -- 주석 컬럼
                , CAST(D.TOTAL_EXCLUDE_SESSION - D.LIVECHAT_SESSION AS DOUBLE) 	AS inbound_count
                -- , NULL					                						AS outbound_count
                , CAST(D.TOTAL_EXCLUDE_SESSION - D.LIVECHAT_SESSION AS DOUBLE) 	AS answered_count
                -- , NULL								                			AS sum_answer_speed
                , CAST(D.SUM_TALK_TIME AS DOUBLE) 								AS sum_talk_time
                -- , CAST(D.SUM_TALK_TIME / D.TOTAL_COUNT AS DOUBLE) 				AS avg_talk_time -- 주석 컬럼
                -- , NULL									                		AS sum_call_work
                -- , NULL									                		AS sum_hold_time
                , CAST(D.SURVEY_SOLVED_SESSION AS DOUBLE) 						AS sum_q1_answer
                , CAST(D.SURVEY_SESSION AS DOUBLE) 								AS sum_q1_response
                -- , CAST(D.SURVEY_SOLVED_SESSION / D.SURVEY_SESSION AS DOUBLE) 	AS counseling_complet_rate -- 주석 컬럼
                , CAST(D.SUM_CHAT_RATING AS DOUBLE) 							AS sum_q2_answer
                , CAST(D.SURVEY_SESSION2 AS DOUBLE) 							AS sum_q2_response
                -- , CAST(D.SUM_CHAT_RATING / D.SURVEY_SESSION2 AS DOUBLE) 		AS satisfaction_score -- 주석 컬럼
                -- , NULL              											AS sum_waiting_time
                -- , NULL			                								AS repeat_call_count
                -- , NULL							                				AS last_agent_conn_count
                -- , NULL					                						AS Ivr_call_count
                -- , NULL								                			AS answered_20s_count
                -- , NULL						                					AS auto_unmanned_ivr_count
                -- , NULL								                			AS agent_login_count
                -- , NULL							                				AS agent_inbound_count
                -- , NULL								                			AS agent_outbound_count
                -- , NULL								                			AS agent_inbound_outbound_count
                -- , NULL								                			AS agent_inbound_lt_5call_count
            from (
                select C.EAP_CORP_CD
                    , C.CREATION_DATE
                    , COUNT(C.CHAT_ID) AS TOTAL_COUNT
                    , SUM(CASE WHEN C.TYPE_SEESION_YN = 'Y' THEN 1 ELSE 0 END) AS TOTAL_EXCLUDE_SESSION
                    , SUM(CASE WHEN C.TYPE_LIVECHAT_YN = 'Y' THEN 1 ELSE 0 END) AS LIVECHAT_SESSION
                    , SUM(C.CHAT_TIME) AS SUM_TALK_TIME
                    , SUM(CASE WHEN C.TYPE_SURVEY_YN = 'Y' THEN 1 ELSE 0 END) AS SURVEY_SESSION
                    , SUM(CASE WHEN C.TYPE_SOLVED_YN = 'Y' THEN 1 ELSE 0 END) AS SURVEY_SOLVED_SESSION
                    , SUM(C.CHAT_RATING) AS SUM_CHAT_RATING
                    , SUM(CASE WHEN C.CHAT_RATING > 0 THEN 1 ELSE 0 END) AS SURVEY_SESSION2
                from (
                    select A.*, B.ATTRIBUTE1 as EAP_CORP_CD, DATE_FORMAT(A.CHAT_DT, '%Y%m%d') as CREATION_DATE
                    from tb_chat_session_info A
                    left join tb_code_mst B
                    on 1=1
                        and B.GROUP_CD = 'B00008'
                        and A.COUNTRY_CD = B.CODE_CD
                ) C
                group by C.EAP_CORP_CD, C.CREATION_DATE
                having 1=1
                -- and C.EAP_CORP_CD IS NOT NULL -- 전체 조회 조건
                and C.EAP_CORP_CD =  %s -- 조회조건
                and C.CREATION_DATE >= %s AND C.CREATION_DATE < %s -- 조회조건
            ) D
            order by 1, 2;
            """
    
    sql_instajce = MySql()
    df = sql_instajce.get_table(query=query, parameters=parameters)
    df = pd.DataFrame(df)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    df.to_excel(f"{corp_cd}_{start_creation_date_utc}_20240603.xlsx",index=False)

############## CORP_CD : LGEAG
############## TIMESTAMP : 2024-06-05T08:30:53
############## START : 20240430
############## END : 20240604
############## CORP_CD : LGEDG
############## TIMESTAMP : 2024-06-05T08:30:53
############## START : 20240430
############## END : 20240604
############## CORP_CD : LGESW
############## TIMESTAMP : 2024-06-05T08:30:53
############## START : 20240531
############## END : 20240604
############## CORP_CD : LGEFS
############## TIMESTAMP : 2024-06-05T08:30:53
############## START : 20240524
############## END : 20240604
############## CORP_CD : LGEIS
############## TIMESTAMP : 2024-06-05T08:30:53
############## START : 20240430
############## END : 20240604
############## CORP_CD : LGEUK
############## TIMESTAMP : 2024-06-05T08:30:53
############## START : 20240529
############## END : 20240604
